# Density analysis script

This notebook will use density files made from the FASTQ processing script and analyze features relevant to translation. This can also be applied to RNA-seq to investigate RNA coverage over the CDS:

   - 

   - Average gene plot condenses ribosome footprints aligned to the start and stop codons of every gene. 
       - Initiation and termination 
       - periodicity
       - occupancy on CDS and UTR
   - Frame analysis condenses reads into the three reading frames, informing on the occupancy at each frame. ribosome footprints should show strong 3nt periodicity. 
       - Frame analyzed using all density sizes, size separated densities, or gene separated 
   - Assymetry analysis informs on the degree of dropoff/runoff in the data. 
   - Pause score analysis provides relative occupancy information on different codons/amino acids. 

# General Settings:

Directory Data:

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime


import ribo_util
import ribo_main
import ribo_analysis
import ribo_plot


organism = 'Coli'      #Coli, Subtilis, Tuberculosis, Salmonella etc...

inputs = {}
inputs['files'] = []
inputs['multiprocess'] = 'yes'
inputs['threads']      = 8   # CPU information for multithreading applications
inputs['cores']        = 4

path_pc     = '/Volumes/HDD/Ribo_seq/'
inpath      = path_pc + 'libraries/'
path_script = '/Users/fuad/Dropbox/'

paths_in = {}
paths_in['path_gff_dict'] = path_pc + 'annotations/'+organism+'/'+organism+'_dict'   #will be made from GFF

paths_out = {}
paths_out['path_density']      = inpath  + 'density/density/'
paths_out['path_log']          = inpath  + 'density/logs/'
paths_out['path_analysis_log'] = inpath  + 'analysis/logs/'
paths_out['path_analysis']     = inpath  + 'analysis/individual/'
paths_out['path_figures']      = inpath  + 'figures/'

'''can place files in .csv with header == Library, or input file names into list  '''
all_files  = []
file_csv   = path_pc  + 'Methods_paper/all_libraries.csv' #path_pc  + 'Methods_paper/all_libraries.csv'
library_id = pd.read_csv(file_csv)

for fname in library_id.Library:
    all_files.append(fname)
    
# Check inputs, create output paths
step = 'density'
ribo_util.check_inputs(inputs, paths_in, step)
ribo_util.createpath(inputs, paths_out)


# Analysis Settings

Set settings values for the various analyses

In [ ]:
# General Settings 
settings = {}
settings['minlength'] = 23
settings['maxlength'] = 35
settings['shift']     = 11
settings['gff_extra'] = 50
settings['threshold'] = 1     # in reads per codon
settings['alignment'] = '3'   # '3' or '5'
# Avggenes Settings
settings['length_out_ORF'] = 50
settings['length_in_ORF']  = 100        # genes shorter than this are excluded
settings['density_type']   = 'reads'    # 'reads' or 'rpm' 
settings['equal_weight']   = 'yes'      # 'yes' or 'no', if yes, change density_type to reads -- faster
settings['next_gene']      = 25         # genes closer than this are removed from start and stop 
settings['threshold']      = 50       # RPKM, genes below will be removed from average

# For asymmetry plot
settings['subgroup'] = '/Volumes/HDD/Ribo_seq/annotations/Coli/SRP_IP.csv'    #provide list of genes or 'none'

# Pausescore settings
settings['A_site shift']    = -11
settings['plot_upstream']   = 40
settings['plot_downstream'] = 50
settings['start_trim']      = 50
settings['stop_trim']       = 20
settings['frameshift']      = 0

# Genelist settings


# Pausescore waves settings
settings['plot_upstream_wave']   = 0
settings['plot_downstream_wave'] = 106
settings['next_codon']           = 'yes'

# Motif analysis
settings['motif_length'] = 9


In [ ]:
lib_count = len(all_files)
lib_index = 0 
lib_runs  = 8  # limits number of samples processed, scale with RAM imitations

for loops in range(0, lib_count / lib_runs + 2):

    if lib_index < lib_count:
        print "Started  at " + str(datetime.now())
        if lib_count - lib_index >= lib_runs:
            inputs['files'] = [all_files[i] for i in range(lib_index, lib_index + lib_runs)]

            print inputs['files']
            
            if not 'gff_dict' in globals(): 
                gff_dict, plus_dict, minus_dict = ribo_util.loadlargePickles(inputs, settings, paths_in, paths_out)
            else: 
                gff_dict, plus_dict, minus_dict = ribo_util.loadlargePickles(inputs, settings, paths_in, paths_out)
            
            average_gene = ribo_analysis.avggenes(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            pausescore_analysis = ribo_analysis.pausescore(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            genelists = ribo_analysis.genelist(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            asymmetry_analysis = ribo_analysis.asymmetry(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            frame_analysis     = ribo_analysis.frame(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            wave_analysis = ribo_analysis.pausescore_waves(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            motif_analysis = ribo_analysis.motif_pausescore(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            
        else:
            inputs['files'] = [all_files[i] for i in range(lib_index, lib_count)]
            
            print inputs['files']    
            if not 'gff_dict' in globals(): 
                gff_dict, plus_dict, minus_dict = ribo_util.loadlargePickles(inputs, paths_in, paths_out)
            else: 
                gff_dict, plus_dict, minus_dict = ribo_util.loadlargePickles(inputs, paths_in, paths_out)
                
            average_gene = ribo_analysis.avggenes(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            pausescore_analysis = ribo_analysis.pausescore(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            genelists = ribo_analysis.genelist(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            asymmetry_analysis = ribo_analysis.asymmetry(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            frame_analysis     = ribo_analysis.frame(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            wave_analysis = ribo_analysis.pausescore_waves(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)
            motif_analysis = ribo_analysis.motif_pausescore(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)

    else:
        continue
    lib_index += lib_runs  
print "Finished  at " + str(datetime.now())

In [ ]:
#for fname in all_files:
    #inputs['files'] = [fname]
    #gff_dict, plus_dict, minus_dict = ribo_util.loadlargePickles(inputs, paths_in, paths_out)

    #motif_analysis = ribo_analysis.motif_pausescore(inputs, paths_out, settings, gff_dict, plus_dict, minus_dict)

# --------------------------------------------------------------------------


# Gene Average Analysis:

Calculate average ribosome occupancy aligned at start or stop codons, as well the symmetry of ribosome distribution on genes and the periodicity associated with the density

Settings for asymmetry analysis:

In [ ]:
inputs['files'] = all_files

settings_plot = {}
settings_plot['HM_max'] = 2
settings_plot['shift']  = 15
settings_plot['ymax']   = 5

average_plot = ribo_plot.plot_avggene(inputs, paths_in, paths_out, settings, settings_plot)

# Frame Analysis

Calculate reading frame of all footprints, as well as for each readlength and each gene:

   - RPKM threshold used above still applies. To change, redefine below 

In [ ]:
frame_plot = ribo_plot.plot_frame(inputs, paths_in, paths_out, settings, settings_plot)

# Pause Score Analysis

Calculate pause scores for each amino acid, and for each codon. 

   - relative density = (reads at position / average reads of the gene)
   - pause score = avg(relative density over codon or aa) / instances of the codon oraa

In [ ]:
settings_plot['aa_or_codon'] = 'aa' 
settings_plot['amino_acid']  = ['P', 'T', 'S', 'I', 'G']
settings_plot['codon']       = ['ATT', 'ATC', 'CCT', 'CCA', 'CCG', 'ACT']

settings_plot['ymax_dot']  = 8
settings_plot['ymax_line'] = 5
settings_plot['vmax_HM']   = 2

plot_pausescore = ribo_plot.plot_pausescore(inputs, paths_in, paths_out, settings, settings_plot)

# Gene Density

Display density of listed genes

   - RPKM threshold used above still applies. To change, redefine below 

# Compare Pausescores

Analyze pausescores using heatmaps:

In [ ]:
paths_in['files']          = path_pc  + 'Methods_paper/all_libraries.csv'
settings_plot['center_HM'] = 1
settings_plot['vmax_HM']   = 3.5

In [ ]:
Pause_heatmap = ribo_plot.plot_pausescore_APE_heatmap(inputs, paths_in, paths_out, settings, settings_plot)

# Serine Wave Plots

Plot ribosome occupancy downstream of serine pauses

In [ ]:
paths_in['files']          = path_pc  + 'Methods_paper/all_libraries.csv'
settings_plot['amino_acid']  = ['S', 'G', 'I']
settings_plot['center_HM']   = 1
settings_plot['vmax_HM']     = 1.5


Pause_wave = ribo_plot.plot_pausescore_downstream(inputs, paths_in, paths_out, settings, settings_plot)

# Asymmetry Plot

In [ ]:
inputs['files'] = all_files
asymmetry = ribo_plot.plot_asymmetry_comp(inputs, paths_in, paths_out, settings)